In [2]:
import pandas as pd

###load all the datasets
##meta_data file
filepath = "/Users/eshwarchandrasekharan/Desktop/buzzed_code/"
meta_df = pd.read_csv(filepath + 'facebook/post_metadata_5-18.csv', index_col=0)
meta_df = meta_df.fillna(0)

##all data file
all_df = pd.read_csv(filepath + 'facebook/post_stats_5-18.csv', index_col=0)
all_df = all_df.fillna(0)

In [94]:
import random
sample_id = random.choice(meta_df['buzz_id'])

In [95]:
meta_df[meta_df.buzz_id == sample_id].sort_values('created_at', ascending = 1)

,external_id,parent_external_id,created_at,page,post_type,buzz_id,uri,title
27893,491452930867938_1570179012995319,0.0,2017-02-25 03:30:06.912389,BuzzFeed Food,link,4471325,restaurant-kitchen-frustrations,22 Struggles Only People Who've Worked In A Re...
28018,21898300328_10155782515165329,0.0,2017-02-25 22:16:10.495083,BuzzFeed,link,4471325,restaurant-kitchen-frustrations,22 Struggles Only People Who've Worked In A Re...
30647,1614251518827491_1881967252055915,0.0,2017-02-28 02:00:06.967880,Tasty,link,4471325,restaurant-kitchen-frustrations,22 Struggles Only People Who've Worked In A Re...
27224,1412984988933193_1958457004385986,0.0,2017-02-28 20:15:11.122585,BuzzFeed Australia,link,4471325,restaurant-kitchen-frustrations,22 Struggles Only People Who've Worked In A Re...
30425,574931839201643_1703853979642751,0.0,2017-03-01 13:09:40.218737,BuzzFeed UK,link,4471325,restaurant-kitchen-frustrations,22 Struggles Only People Who've Worked In A Re...


In [13]:
for title in meta_df[meta_df.buzz_id == sample_id].title:
    print(title)

The Nominee Who Won't Be At The Oscars Is The One Who Matters Most
The Nominee Who Won't Be At The Oscars Is The One Who Matters Most
The Nominee Who Won't Be At The Oscars Is The One Who Matters Most
The Nominee Who Won't Be At The Oscars Is The One Who Matters Most


In [15]:
meta_df[meta_df.title== "We'll Reveal When You're Going To Die, But First Choose Your Last Meal On Earth"]

,external_id,parent_external_id,created_at,page,post_type,buzz_id,uri,title
54510,560066014088988_1333195940109321,0.0,2017-05-20 17:30:41.155736,BuzzFeed Quiz,link,4530023,the-last-supper,"We'll Reveal When You're Going To Die, But Fir..."
54821,491452930867938_1670316296314923,0.0,2017-05-22 14:48:19.863178,BuzzFeed Food,link,4530023,the-last-supper,"We'll Reveal When You're Going To Die, But Fir..."


In [43]:
### Which pages co-occur the most? What's the most common cross-promoting pattern?
cooccur = {}

for link_id in meta_df.buzz_id.unique():
#     print(link_id)
    temp = meta_df[meta_df.buzz_id == link_id]
    temp = temp.sort_values('created_at', ascending = 1)
    pages = temp.page.unique()
    for page in pages:
        crosspost = pages
        ###drop page from crosspost
        try:
            cooccur[page].append(crosspost)
        except:
            cooccur[page] = crosspost

In [68]:
l = {}
for key in cooccur:
    l[key] = len(cooccur[key])
max(l, key=l.get)

'SOML'

In [76]:
search_copost = 1
loners = []
for key in l:
    if l[key] == search_copost:
        loners.append(key)
        
len(loners), loners

(45,
 ['BuzzFeed Books',
  'BuzzFeed España',
  'BuzzFeed Scotland',
  'BuzzFeed Style',
  'BuzzFeed Parents',
  'BuzzFeed News BR',
  'BuzzFeed News',
  'BuzzFeed Celeb',
  'BuzzFeed DIY',
  'LOLA',
  'BuzzFeed Politics',
  'BuzzFeed World',
  'BuzzFeed UK News',
  'Cheeky',
  'So Relatable',
  'BuzzFeed France',
  'BuzzFeed Philippines',
  'BuzzFeed Health',
  'BuzzFeed Deutschland',
  'BuzzFeed Rewind',
  'BuzzFeed Animals',
  'BuzzFeed Reader',
  'BuzzFeed Español',
  'BuzzFeed India',
  'BuzzFeed Brasil',
  'BuzzFeed Oz Politics',
  'BuzzFeed France News',
  'BuzzFeed Tech',
  'BuzzFeed Geeky',
  'BuzzFeed México',
  'Quizzes En Español',
  'Pero Like',
  'BuzzFeed UK Politics',
  'BuzzFeed Open Lab',
  'Bien Tasty',
  'BuzzFeed Kevin N.',
  'Nifty Science',
  'BuzzFeed Japan',
  'BuzzFeed Japan News',
  'BuzzFeed Partner',
  'Oh Great, More Politics',
  'BuzzFeed Pink',
  'See Something Say Something',
  'Tasty One-Pot',
  'Tasty Japan'])

In [177]:
####top 1% of most clicked/shared articles - how many of these were cross-promoted/posted?

import pandas as pd
from pandas.stats.api import ols
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
%matplotlib inline

pages = ['BuzzFeed Books', 'Tasty', 'BuzzFeed España', 'BuzzFeed Scotland', 'SOML', 'BuzzFeed News BR', 'Proper Tasty',]
# page_info = pd.read_csv('datapoints/data-points-Tasty.csv')

for page in pages:
    page_info = pd.read_csv(filepath + 'datapoints/data-points-' + page + '.csv')
    ###SHARES
    print("Page = ",page)
    print(page + ": Top 100 shared articles!")
    top100_clicks = page_info.sort_values('24_hours_share', ascending = False)[:100]['external_id']

    crosspost_info = pd.read_csv(filepath+'facebook/post_metadata_5-18.csv')
    crosspromotions = []

    for link in top100_clicks:
        buzzid = crosspost_info[crosspost_info.external_id == link].buzz_id.values[0]
        temp = crosspost_info[crosspost_info.buzz_id == buzzid]
#         print (link, " : ", buzzid, " ; crosspromoted ", len(temp), " time(s)!")
        crosspromotions.append(len(temp))
    #     temp = temp.sort_values('created_at', ascending = 1)
    #     pages = temp.page.unique()
    #     for page in pages:
    #         crosspost = pages
    #         ###drop page from crosspost
    #         try:
    #             cooccur[page].append(crosspost)
    #         except:
    #             cooccur[page] = crosspost
    print(max(crosspromotions), min(crosspromotions), sum(crosspromotions)/len(crosspromotions))
    count1 = 0
    for i in crosspromotions:
        if i == 1:
            count1 +=1
    print(count1)

    ###CLICKS
    print(page + ": Top 100 clicked articles!")
    top100_clicks = page_info.sort_values('24_hours_clicks', ascending = False)[:100]['external_id']

    crosspost_info = pd.read_csv(filepath+'facebook/post_metadata_5-18.csv')
    crosspromotions = []

    for link in top100_clicks:
        buzzid = crosspost_info[crosspost_info.external_id == link].buzz_id.values[0]
        temp = crosspost_info[crosspost_info.buzz_id == buzzid]
#         print (link, " : ", buzzid, " ; crosspromoted ", len(temp), " time(s)!")
        crosspromotions.append(len(temp))
    #     temp = temp.sort_values('created_at', ascending = 1)
    #     pages = temp.page.unique()
    #     for page in pages:
    #         crosspost = pages
    #         ###drop page from crosspost
    #         try:
    #             cooccur[page].append(crosspost)
    #         except:
    #             cooccur[page] = crosspost
    print(max(crosspromotions), min(crosspromotions), sum(crosspromotions)/len(crosspromotions))
    count1 = 0
    for i in crosspromotions:
        if i == 1:
            count1 +=1
    print(count1)

Page =  BuzzFeed Books
BuzzFeed Books: Top 100 shared articles!
12 1 4.26
17
BuzzFeed Books: Top 100 clicked articles!
12 1 4.77
11
Page =  Tasty
Tasty: Top 100 shared articles!
11 2 5.49
0
Tasty: Top 100 clicked articles!
12 2 6.07
0
Page =  BuzzFeed España
BuzzFeed España: Top 100 shared articles!
4 1 1.83
40
BuzzFeed España: Top 100 clicked articles!
4 1 1.94
32
Page =  BuzzFeed Scotland
BuzzFeed Scotland: Top 100 shared articles!
11 1 2.29
39
BuzzFeed Scotland: Top 100 clicked articles!
11 1 2.14
39
Page =  SOML
SOML: Top 100 shared articles!
11 1 5.74
3
SOML: Top 100 clicked articles!
11 1 5.78
2
Page =  BuzzFeed News BR
BuzzFeed News BR: Top 100 shared articles!
2 1 1.01
99
BuzzFeed News BR: Top 100 clicked articles!
2 1 1.05
95
Page =  Proper Tasty
Proper Tasty: Top 100 shared articles!
6 2 3.75
0
Proper Tasty: Top 100 clicked articles!
6 2 3.75
0


(12, 1, 4.77)

In [173]:
count1 = 0
for i in crosspromotions:
    if i == 1:
        count1 +=1
print(count1)

11
